In [1]:
import pika
import pandas as pd
from io import StringIO


In [2]:
# Initialize an empty DataFrame
df = pd.DataFrame()


In [3]:
# RabbitMQ connection parameters
rabbitmq_host = 'localhost'
queue_name = 'my_queue'


In [4]:
def callback(ch, method, properties, body):
    global df
    # Assuming the body is a string of comma-separated values
    message = body.decode('utf-8')
    # Convert the message to a file-like object
    message_io = StringIO(message)
    # Create a DataFrame from the message
    message_df = pd.read_csv(message_io, header=None)
    # Append the message DataFrame to the main DataFrame
    df = pd.concat([df, message_df], ignore_index=True)
    print(df)
    

In [5]:
# Connect to RabbitMQ
connection = pika.BlockingConnection(pika.ConnectionParameters(host=rabbitmq_host))
channel = connection.channel()

# Make sure the queue exists and is durable
channel.queue_declare(queue=queue_name, durable=True)

# Set up the subscription on the queue
channel.basic_consume(queue=queue_name, on_message_callback=callback, auto_ack=True)

print(' [*] Waiting for messages. To exit press CTRL+C')

# Start consuming messages from the queue
channel.start_consuming()

 [*] Waiting for messages. To exit press CTRL+C
   0  1  2
0  a  b  c
   0  1  2
0  a  b  c
1  a  b  c
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
3  g  h  i
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
3  g  h  i
4  a  b  c
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
3  g  h  i
4  a  b  c
5  a  b  c
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
3  g  h  i
4  a  b  c
5  a  b  c
6  d  e  f
   0  1  2
0  a  b  c
1  a  b  c
2  d  e  f
3  g  h  i
4  a  b  c
5  a  b  c
6  d  e  f
7  g  h  i


AMQPHeartbeatTimeout: No activity or too many missed heartbeats in the last 60 seconds